# General Imports

In [3]:
from sklearn.datasets import make_regression
import numpy as np
import warnings

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.inspection import permutation_importance

import subprocess
import sys

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt


Disable warnings

In [4]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

In [5]:
# Generate regression toy data
n_samples = 1000
n_features = 5
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42)

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)

X_train = X[:train_size]
y_train = y[:train_size]

# Test data
X_test = X[train_size:]
y_test = y[train_size:]

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [7]:

#results = dt_bo.maximize(n_iter=5, init_points=20)

In [8]:
all_models = {}

# MLR

**feature selection**

In [9]:
# Assuming you have X_train and y_train defined for training data

# Initialize the Linear Regression model
lr_model = LinearRegression()

# Fit the Linear Regression model to the training data
lr_model.fit(X_train, y_train)

# Get feature coefficients
coefficients = lr_model.coef_

# Create a list of feature names or indices paired with their coefficients
feature_coefficients = list(zip(range(X_train.shape[1]), coefficients))

# Sort the features based on absolute coefficient values in descending order
feature_coefficients.sort(key=lambda x: abs(x[1]), reverse=True)

# Print the ranked feature coefficients
print("Feature Coefficients:")
for feature_index, coefficient in feature_coefficients:
    print(f"Feature {feature_index}: {coefficient}")

Feature Coefficients:
Feature 2: 46.07121713482753
Feature 3: 28.6279862111941
Feature 4: 24.74629812331462
Feature 1: 18.993474366101992
Feature 0: 16.823657910849178


**Hyperparameter optimalisatie**
#heb deze even uitgecomment omdat hij breekt als er geen min en max values zijn gedefinieerd

In [10]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

# Define the MLR model evaluation function using cross-validation
def evaluate_mlr_model(fit_intercept):
    # Convert fit_intercept to a boolean value
    fit_intercept = bool(fit_intercept)
    
    # Create and configure the MLR model
    model = LinearRegression(fit_intercept=fit_intercept)
    
    # Perform cross-validation on the training data
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    # Return the negative mean squared error (Bayesian Optimization maximizes the objective)
    return np.mean(scores)

# Define the parameter ranges for Bayesian Optimization
params_ranges = {
    'fit_intercept': (0, 1)
}

# Perform Bayesian Optimization
mlr_bo = BayesianOptimization(f=evaluate_mlr_model, pbounds=params_ranges)
mlr_bo.maximize(n_iter=10, init_points=5)

# Get the best hyperparameters
best_params = mlr_bo.max['params']
best_fit_intercept = bool(best_params['fit_intercept'])

# Create the best MLR model with the tuned hyperparameters
best_model_mlr = LinearRegression(fit_intercept=best_fit_intercept)

# Fit the best model to the training data
best_model_mlr.fit(X_train, y_train)

# Calculate evaluation metrics
y_pred = best_model_mlr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
aed = np.abs(y_test.mean() - y_pred)
r2 = r2_score(y_test, y_pred)
n = len(X_test)
k = X_test.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("AED:", aed)
print("R2:", r2)
print("Adjusted R2:", r2_adj)

|   iter    |  target   | fit_in... |
-------------------------------------
| 1         | -2.967e-2 | 0.1723    |
| 2         | -2.967e-2 | 0.2472    |
| 3         | -2.967e-2 | 0.1304    |
| 4         | -2.967e-2 | 0.8852    |
| 5         | -2.967e-2 | 0.1051    |
| 6         | -2.967e-2 | 0.9999    |
| 7         | -2.967e-2 | 1.0       |
| 8         | -2.967e-2 | 0.1476    |
| 9         | -2.967e-2 | 0.9999    |
| 10        | -2.967e-2 | 8.382e-05 |
| 11        | -2.967e-2 | 0.9999    |
| 12        | -2.967e-2 | 0.1246    |
| 13        | -2.967e-2 | 0.7575    |
| 14        | -2.967e-2 | 0.9728    |
| 15        | -2.967e-2 | 0.9961    |
RMSE: 3.374228512561674e-14
MSE: 1.1385418054984168e-27
MAE: 2.5738855491397318e-14
AED: [139.85097425   2.77284493  37.42315787 103.77347117  85.13420909
  35.81586103  16.90197153 108.51327206  36.3560781   87.96182873
  61.54456018  48.76570676 154.28984418   9.88074801   2.31632614
  26.85400903   5.53414449  76.04103573  70.23431223   9.06672153
 

# Support Vector Machines

**Feature selection**

In [11]:
# Assuming you have X_train and y_train defined for training data

# Initialize the SVM model
svm_model = SVR(kernel='rbf')  # Replace 'rbf' with your desired kernel

# Fit the SVM model to the training data
svm_model.fit(X_train, y_train)

# Compute permutation importances
result = permutation_importance(svm_model, X_train, y_train, n_repeats=10, random_state=42)

# Get feature importances
importances = result.importances_mean

# Create a list of feature names or indices paired with their importances
feature_importances = list(zip(range(X_train.shape[1]), importances))

# Sort the features based on importance in descending order
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Print the ranked feature importances
print("Feature Importances:")
for feature_index, importance in feature_importances:
    print(f"Feature {feature_index}: {importance}")

Feature Importances:
Feature 2: 0.5100907245192264
Feature 3: 0.1812236125303029
Feature 4: 0.13843292474891247
Feature 1: 0.10140750241353605
Feature 0: 0.07508208211413692


**Hyperparameter optimalisatie**

In [12]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")
 
# Define the SVM model evaluation function using cross-validation
def evaluate_svm_model(C, epsilon, gamma):
    # Create and configure the SVM model
    model = SVR(C=C, epsilon=epsilon, gamma=gamma)
    
    # Perform cross-validation on the training data
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    # Return the negative mean squared error (Bayesian Optimization maximizes the objective)
    return np.mean(scores)

# Define the parameter ranges for Bayesian Optimization
params_ranges = {
    'C': (0.1, 10),
    'epsilon': (0.01, 1),
    'gamma': (0.001, 0.1)
}

# Perform Bayesian Optimization
svm_bo = BayesianOptimization(f=evaluate_svm_model, pbounds=params_ranges)
svm_bo.maximize(n_iter=10, init_points=5)

# Get the best hyperparameters
best_params = svm_bo.max['params']
best_C = best_params['C']
best_epsilon = best_params['epsilon']
best_gamma = best_params['gamma']

# Create the best SVM model with the tuned hyperparameters
best_model_svm = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma)

# Fit the best model to the training data
best_model_svm.fit(X_train, y_train)

# Calculate evaluation metrics
y_pred = best_model_svm.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
aed = np.abs(y_test.mean() - y_pred)
r2 = r2_score(y_test, y_pred)
n = len(X_test)
k = X_test.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("AED:", aed)
print("R2:", r2)
print("Adjusted R2:", r2_adj)

|   iter    |  target   |     C     |  epsilon  |   gamma   |
-------------------------------------------------------------
| 1         | -323.5    | 4.361     | 0.447     | 0.09398   |
| 2         | -42.21    | 6.607     | 0.2209    | 0.02074   |
| 3         | -1.775e+0 | 2.041     | 0.7572    | 0.013     |
| 4         | -3.201e+0 | 7.644     | 0.3761    | 0.001076  |
| 5         | -299.9    | 4.207     | 0.6755    | 0.01334   |
| 6         | -80.27    | 5.944     | 0.2128    | 0.03218   |
| 7         | -281.0    | 6.184     | 0.9899    | 0.09704   |
| 8         | -249.4    | 5.112     | 0.9823    | 0.07812   |
| 9         | -3.766e+0 | 0.1       | 0.01      | 0.1       |
| 10        | -209.1    | 10.0      | 1.0       | 0.1       |
| 11        | -208.1    | 10.0      | 0.01      | 0.1       |
| 12        | -389.2    | 3.315     | 0.01      | 0.1       |
| 13        | -390.1    | 3.276     | 1.0       | 0.1       |
| 14        | -222.5    | 9.228     | 1.0       | 0.1       |
| 15    

# Random Forest

**Feature selection**

In [13]:
# Assuming you have X_train and y_train defined for training data

# Initialize the Random Forest model
rf_model = RandomForestRegressor()

# Fit the Random Forest model to the training data
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_

# Create a list of feature names or indices paired with their importances
feature_importances = list(zip(range(X_train.shape[1]), importances))

# Sort the features based on importance in descending order
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Print the ranked feature importances
print("Feature Importances:")
for feature_index, importance in feature_importances:
    print(f"Feature {feature_index}: {importance}")


Feature Importances:
Feature 2: 0.5501316433194556
Feature 3: 0.17443755735216024
Feature 4: 0.13315391769428214
Feature 1: 0.08445412244275924
Feature 0: 0.05782275919134279


**Hyperparameter optimalisatie**

In [14]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

params_ranges = {
    'n_estimators': (10, 100),
    'max_depth': (1, 20),
    'min_samples_leaf': (1, 10),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_features': (0.1, 1),
    'max_leaf_nodes': (10, 100)
}

# Example usage with Random Forest
model = RandomForestRegressor
dt_bo = BayesianOptimization(f=lambda n_estimators, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'n_estimators': int(round(n_estimators)),
                                        'max_depth': int(round(max_depth)),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': int(round(max_leaf_nodes))
                                    }, X_train, y_train),
                             pbounds=params_ranges)
results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model_random_forest = model(
    n_estimators=int(round(params['n_estimators'])),
    max_depth=int(round(params['max_depth'])),
    min_samples_leaf=round(params['min_samples_leaf']),
    min_weight_fraction_leaf=params['min_weight_fraction_leaf'],
    max_features=params['max_features'],
    max_leaf_nodes=int(round(params['max_leaf_nodes']))
)

# Fit the model
best_model_random_forest.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model_random_forest.predict(X_test)

mse_scores = mean_squared_error(y_test, y_pred)
mae_scores = mean_absolute_error(y_test, y_pred)
aed_scores = np.abs(y_test.mean() - y_pred)
r2_scores = r2_score(y_test, y_pred)

n = len(X_test)
k = X_test.shape[1]
r2_adj_scores = 1 - (1 - r2_scores) * ((n - 1) / (n - k - 1))
rmse_scores = np.sqrt(mse_scores)

print("MSE:", mse_scores)
print("MAE:", mae_scores)
print("AED:", aed_scores)
print("R2:", r2_scores)
print("Adjusted R2:", r2_adj_scores)
print("RMSE:", rmse_scores)


|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 50.63     | 8.121     | 0.9859    | 34.06     | 4.935     | 0.4064    | 19.51     |
| 2         | 50.53     | 2.032     | 0.9123    | 55.0      | 7.922     | 0.3517    | 40.33     |
| 3         | 50.93     | 4.83      | 0.6749    | 56.82     | 7.786     | 0.3364    | 84.48     |
| 4         | 45.18     | 12.38     | 0.7898    | 72.2      | 9.941     | 0.2326    | 84.05     |
| 5         | 39.14     | 13.66     | 0.602     | 31.43     | 3.796     | 0.1262    | 20.29     |
| 6         | 41.96     | 9.239     | 0.8278    | 18.48     | 6.067     | 0.1676    | 22.79     |
| 7         | 51.31     | 16.91     | 0.6602    | 41.87     | 6.485     | 0.4579    | 78.01     |
| 8         | 53.0      | 1.023     | 0.4364    | 10.15     | 3.408     | 0.4169    | 90.38     |
| 9         | 43.13 

# neural network

**Feature selection**

In [15]:
# Assuming you have X and y defined for the dataset

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the neural network regressor
nn_model = MLPRegressor(hidden_layer_sizes=(10, 10))  # Adjust the architecture as needed

# Fit the neural network model to the training data
nn_model.fit(X_train_scaled, y_train)

# Perform feature selection using Recursive Feature Elimination (RFE)
selector = RFE(estimator=nn_model, n_features_to_select=10)  # Adjust n_features_to_select as needed
selector.fit(X_train_scaled, y_train)

# Transform the training and testing sets to keep only the selected features
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

# Print the selected feature support
selected_support = selector.support_
print("Selected Feature Support:")
print(selected_support)

Selected Feature Support:
[ True  True  True  True  True]


**Hyperparameter optimalisatie**

In [16]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000),
}

# Example usage with Neural Network
model = MLPRegressor
dt_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model_neural_network = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)


# Fit the model
best_model_neural_network.fit(X_train, y_train)


# Calculate the evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
aed = np.abs(y_test.mean() - y_pred)
r2 = r2_score(y_test, y_pred)
n = len(X_test)
k = X_test.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("AED:", aed)
print("R2:", r2)
print("Adjusted R2:", r2_adj)


|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | 0.3072    | 0.04584   | 19.78     | 0.01561   | 859.4     |
| 2         | 0.2219    | 0.04201   | 81.93     | 0.03815   | 732.0     |
| 3         | 0.2146    | 0.009361  | 62.17     | 0.06643   | 395.4     |
| 4         | 0.239     | 0.05724   | 65.39     | 0.02733   | 649.9     |
| 5         | 0.3509    | 0.01487   | 41.03     | 0.0179    | 460.6     |
| 6         | 0.2575    | 0.06477   | 65.03     | 0.03483   | 198.0     |
| 7         | 0.1973    | 0.0668    | 26.0      | 0.08392   | 318.2     |
| 8         | 0.2569    | 0.0645    | 50.05     | 0.04003   | 468.7     |
| 9         | 0.2839    | 0.08144   | 69.25     | 0.01969   | 962.4     |
| 10        | 0.3689    | 0.02508   | 60.83     | 0.012     | 550.0     |
| 11        | 8.614     | 0.08342   | 76.13     | 0.001021  | 217.4     |
| 12        | 0.1857    | 0.04557   | 

In [17]:
# Append model to list
all_models['Random forest'] = best_model_random_forest
all_models['SVM'] = best_model_svm
all_models['Neural Network'] = best_model_neural_network
all_models['MLR']= best_model_mlr

# plotting the bar chart

**using train dataset**

In [18]:
# Evaluation metric labels
metric_labels = ['RMSE', 'MSE', 'MAE', 'AED', 'R2', 'Adjusted R2']

# Calculate evaluation metrics using cross-validation for each model
metrics = {
    'RMSE': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    'MSE': mean_squared_error,
    'MAE': mean_absolute_error,
    'AED': lambda y_true, y_pred: np.abs(np.mean(y_true) - y_pred),
    'R2': r2_score,
    'Adjusted R2': lambda y_true, y_pred: 1 - ((1 - r2_score(y_true, y_pred)) * (len(y_true) - 1) / (len(y_true) - X_train.shape[1] - 1))
}

model_names = ['Random Forest', 'SVM', 'Neural Network', 'MLR']

models = {
    'Random Forest': best_model_random_forest,
    'SVM': best_model_svm,
    'Neural Network': best_model_neural_network,
    'MLR': best_model_mlr
}

metric_scores = {metric: [] for metric in metric_labels}

for model_name in model_names:
    model = models[model_name]
    y_pred = cross_val_predict(model, X_train, y_train, cv=5)
    for metric in metric_labels:
        metric_scores[metric].append(metrics[metric](y_train, y_pred))

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()


**using test dataset**

In [19]:

metric_labels = ['RMSE', 'MSE', 'MAE', 'AED', 'R2', 'Adjusted R2']

metrics = {
    'RMSE': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    'MSE': mean_squared_error,
    'MAE': mean_absolute_error,
    'AED': lambda y_true, y_pred: np.abs(np.mean(y_true) - y_pred),
    'R2': r2_score,
    'Adjusted R2': lambda y_true, y_pred: 1 - ((1 - r2_score(y_true, y_pred)) * (len(y_true) - 1) / (len(y_true) - X_train.shape[1] - 1))
}

model_names = ['Random Forest', 'SVM', 'Neural Network', 'MLR']

models = {
    'Random Forest': best_model_random_forest,
    'SVM': best_model_svm,
    'Neural Network': best_model_neural_network,
    'MLR': best_model_mlr
}

# Train each model on the training data and predict the test data
predictions = {}
for model_name in model_names:
    model = models[model_name]
    model.fit(X_train, y_train)  # Train the model
    predictions[model_name] = model.predict(X_test)  # Predict the test data

# Calculate each metric for each model's predictions
metric_scores = {metric: [] for metric in metric_labels}
for model_name in model_names:
    y_pred = predictions[model_name]
    for metric in metric_labels:
        metric_scores[metric].append(metrics[metric](y_test, y_pred))  # Use the test data here

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()